In [ ]:
from openai import AzureOpenAI


azure_openai = AzureOpenAI(
    azure_endpoint="<YOUR_AZURE_OPENAI_ENDPOINT>",
    api_key="<YOUR_AZURE_OPENAI_API_KEY>",
    api_version="2025-01-01-preview",
)


import time

start_time = time.time()
response = azure_openai.chat.completions.create(
    model="gpt-4.1",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello, how are you?"},
    ],
    temperature=0.1,
    max_tokens=100,
    top_p=0.1,
    stream=True,
)


for i, chunk in enumerate(response):
    if i == 0:
        first_token_time = time.time()
        break

# 결과 출력
ttft = first_token_time - start_time
print(f"⏱️ Time to First Token (TTFT): {ttft:.3f} seconds")

In [ ]:
import timeit

def wrapper():
    start = time.time()
    for i, chunk in enumerate(azure_openai.chat.completions.create(
        model="gpt-4.1",
        messages=[{"role": "user", "content": "Hi"}],
        stream=True
    )):
        if i == 0:
            break
    return time.time() - start

# 10번 실행
latencies = timeit.repeat(wrapper, repeat=10, number=1)
print(f"Avg: {sum(latencies)/len(latencies):.3f}, min: {min(latencies):.3f}, max: {max(latencies):.3f}")

In [ ]:
import time
import openai

total_tokens = 0
first_token_time = None
last_token_time = None

response = azure_openai.chat.completions.create(
    model="gpt-4.1",
    messages=[{"role": "user", "content": "Explain quantum computing in simple terms."}],
    stream=True
)

for i, chunk in enumerate(response):
    if i == 0:
        first_token_time = time.time()
    last_token_time = time.time()

    # 토큰 하나 도착하면 1개 증가 (정확한 추정)
    if not chunk.choices:
        continue
    delta = chunk.choices[0].delta
    if delta.content:
        total_tokens += 1

completion_time = last_token_time - first_token_time
tokens_per_second = completion_time / total_tokens if completion_time > 0 else 0

print(f"Completion Time: {completion_time:.2f} sec")
print(f"Tokens Generated: {total_tokens}")
print(f"Tokens per Second: {tokens_per_second:.4f}")

In [ ]:
%pip install -U "autogen-agentchat" "autogen-ext[openai]"

In [ ]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

async def main() -> None:
    model_client = AzureOpenAIChatCompletionClient(
        model="gpt-4.1",
        api_key="<YOUR_AZURE_OPENAI_API_KEY>",
        azure_endpoint="<YOUR_AZURE_OPENAI_ENDPOINT>",
        api_version="2025-01-01-preview",
    )
    agent = AssistantAgent("assistant", model_client=model_client)
    print(await agent.run(task="Who are you?"))
    await model_client.close()

await main()